### Парсинг данных с сайта авто.ру

In [ ]:
import pandas as pd
import pickle
from bs4 import BeautifulSoup
import requests
import time
from multiprocessing.dummy import Pool as ThreadPool
import re
from multiprocessing import Process, Lock, Value
import json
import numpy as np
pd.set_option('display.max_columns', 500) 
from tqdm import tqdm
from pandas_profiling import ProfileReport
from collections import Counter

from datetime import timedelta, datetime, date

import pandas.api.types as ptypes

### Открываем тестовый датафрейм

In [52]:
test = pd.read_csv('test.csv')

### Создаем список ссылок машин из тестового датафрейма

In [7]:
firsPageUrl = test['car_url'].to_list()
f = open("testFirsPageUrl.pkl","wb")
pickle.dump(firsPageUrl,f)
f.close()

### Парсинг по ссылкам проданных машин для получения ссылок похожих машин на проданные

In [7]:
start = Value('I',0)
def getTheSame(url):
    sameCars = []
    r = requests.get(url)
    soup = BeautifulSoup(r.content)
    div = soup.find("div", class_="CardBreadcrumbs")
    try: 
        with start.get_lock():
            start.value+=1
            print(start.value)
        sameCars.append(div.find_all('a')[-1]['href'])
        print(url, 'parsed')
        return sameCars
    except AttributeError:  
        print("No CardBreadcrumbs call found") 

In [ ]:
# Применяем мультипроцессинг для ускоренного парсинга
pool = ThreadPool()
result = pool.map(getTheSame, firsPageUrl)

In [28]:
# Трансформируем полученый список списков в плоский список
flat_results = []
for i in result:
    if i != None:
        flat_results.append(i[0])

In [29]:
# Проверяем колличество уникальных линков в списке. У нас 5842 разновидности машин
print(len(flat_results))
flat_results = list(set(flat_results))
results = flat_results
print(len(flat_results))

34451
5842


In [14]:
# Сохраняем полученый список локально
with open("sameCarsURLs.txt", "wb") as fp:
    pickle.dump(results, fp)

In [15]:
# Открываем список из локального файла
with open("sameCarsURLs.txt", "rb") as fp:  
    sameCarsURLs = pickle.load(fp)

In [23]:
len(sameCarsURLs)

5842

### Второй парсинг - получение ссылок на похожие машины

In [9]:
def carsInList(url1):
    carsURLs = []
    r = requests.get(url1)
    soup = BeautifulSoup(r.text, 'html.parser')
    div = soup.select('.ListingItem-module__main .ListingItem-module__thumb a')
    with start.get_lock():
            start.value+=1
            print(start.value)
    for i in div:
        carsURLs.append(i['href'])
    return carsURLs

In [ ]:
# Запускаем мультипроцессиг
start = Value('I',0)
pool = ThreadPool()
results = pool.map(carsInList, sameCarsURLs)

In [21]:
#  Сохраняем полученый список локально
with open("sameCarsURLsSecond.txt", "wb") as fp:
    pickle.dump(results, fp)

In [22]:
# делаем список плоским и проверяем наличие уникальных значений
flat_results_2 = []
for i in results:
    for a in i:
        flat_results_2.append(a)
len(set(flat_results_2))

68890

### Поиск машин по Москве

In [12]:
# Список брендов
brand = list(test.brand.unique())

In [20]:
def parsing_url(brand):
    carsURLs = [] # здесь соберем список списков(эл-т - список параметров конкретного автомобиля)
    for i in range(1, 94):     # кол-во страниц для парсинга с авто.ру   
        response = requests.get(f'https://auto.ru/moskva/cars/{brand}/used/?output_type=list&page={i}')
        print(f'{brand} page={i}')
        with start.get_lock():
            start.value+=1
            print(start.value)
        if response.status_code != 200:
            raise BaseException("response code" + str(response.status_code))
        soup = BeautifulSoup(response.text, 'html.parser')
        div = soup.select('.ListingItem-module__main .ListingItem-module__thumb a')
            
        for j in div:
            carsURLs.append(j['href'])
    return carsURLs

In [21]:
# Запескаем мультипроцессинг

start = Value('I',0)
pool = ThreadPool()
moskowCars = pool.map(parsing_url, brand)

VOLVO page=99
1
AUDI page=99
2
INFINITI page=99
3
HONDA page=99
4NISSAN page=99SKODA page=99


5
6
MERCEDES page=99
7
BMW page=99
8
VOLVO page=100
9
HONDA page=100
10
SKODA page=100
11
NISSAN page=100
12
INFINITI page=100
13
AUDI page=100
14
MERCEDES page=100
15
BMW page=100
16
VOLVO page=101
17
HONDA page=101
18
MERCEDES page=101
19
AUDI page=101
20
SKODA page=101
21
VOLVO page=102
22
INFINITI page=101
23
NISSAN page=101
24
BMW page=101
25
HONDA page=102
26
MERCEDES page=102
27
VOLVO page=103
28
SKODA page=102
29
AUDI page=102
30
NISSAN page=102
31
INFINITI page=102
32
HONDA page=103
33
BMW page=102
34
VOLVO page=104
35
MERCEDES page=103
36
SKODA page=103
37
AUDI page=103
38
NISSAN page=103
39
HONDA page=104
40
BMW page=103
41
INFINITI page=103
42
MERCEDES page=104
43
VOLVO page=105
44
HONDA page=105
45
SKODA page=104
46
NISSAN page=104
47
AUDI page=104
48
BMW page=104
49
MERCEDES page=105
50
INFINITI page=104
51
VOLVO page=106
52
HONDA page=106
53
NISSAN page=105
54
SKODA page=105
55

In [22]:
# Делаем плоский список

flat_results = []
for i in moskowCars:
    for j in i:
        flat_results.append(j)

In [19]:
with open("allMoscowCarsURLsFrom95.txt", "wb") as fp:
    pickle.dump(flat_results, fp)

In [25]:
with open("allMoscowCarsURLs.txt", "rb") as fp:  
    mskCarsURLs = pickle.load(fp)

In [26]:
# Проверяем длину списока машин проланых в Москве
print(len(mskCarsURLs))
print(len(list(set(mskCarsURLs))))

28179
28074


### Третий парсинг машин и создание датафрейма

In [7]:
df_train = pd.DataFrame() 

for url in tqdm(flat_results_2[]):
    try:
        res = requests.get(url)
        res.encoding = 'utf-8'
        soup = BeautifulSoup(res.text, 'html.parser')
        
        script1 = soup.find('script', type='application/ld+json').contents[0]
        script1_json = json.loads(script1)
        script2 = soup.find('script', id='initial-state').contents[0]
        script2_json = json.loads(script2)
        ul = soup.find('ul', class_='CardInfo')
    except:
        continue
        
    train = {}

    try:
        train['bodyType'] = script1_json['bodyType']   
    except:
        np.nan        
    try:
        train['brand'] = script1_json['brand']
    except:
        np.nan   
    try:
        train['car_url'] = script1_json['offers']['url']
    except:
        np.nan   
    try:
        train['color'] = script1_json['color']
    except:
        np.nan   
    try:
        train['complectation_dict'] = np.nan
    except:
        np.nan    
    try:
        train['description'] = script1_json['description']
    except:
        np.nan    
    try:
        train['engineDisplacement'] = script1_json['vehicleEngine']['engineDisplacement']
    except:
        np.nan   
    try:
        train['enginePower'] = script1_json['vehicleEngine']['enginePower']
    except:
        np.nan   
    try:
        train['equipment_dict'] = str(script2_json['card']['vehicle_info']['equipment'])
    except:
        np.nan    
    try:
        train['fuelType'] = script1_json['vehicleEngine']['fuelType']
    except:
        np.nan   
    try:
        train['image'] = script1_json['image']
    except:
        np.nan    
    try:
        train['mileage'] = script2_json['card']['state']['mileage']
    except:
        np.nan   
    try:
        train['modelDate'] = script1_json['modelDate']
    except:
        np.nan    
    try:
        train['model_info'] = str(script2_json['card']['vehicle_info']['model_info'])
    except:
        np.nan   
    try:
        train['model_name'] = script2_json['card']['vehicle_info']['model_info']['code']
    except:
        np.nan
        
    try:
        train['name'] = script2_json['card']['vehicle_info']['tech_param']['human_name']
    except:
        np.nan    
    try:
        train['numberOfDoors'] = script1_json['numberOfDoors']
    except:
        np.nan    
    try:
        train['parsing_unixtime'] = int(time.time())
    except:
        np.nan    
    try:
        train['priceCurrency'] = script1_json['offers']['priceCurrency']
    except:
        np.nan    
    try:
        train['productionDate'] = script1_json['productionDate']
    except:
        np.nan    
    try:
        train['sell_id'] = script2_json['card']['id']
    except:
        np.nan    
    try:
        train['super_gen'] = str(script2_json['card']['vehicle_info']['tech_param'])
    except:
        np.nan    
    try:
        train['vehicleConfiguration'] = script1_json['vehicleConfiguration']
    except:
        np.nan    
    try:
        train['vehicleTransmission'] = script1_json['vehicleTransmission']
    except:
        np.nan    
    try:
        train['vendor'] = np.nan
    except:
        np.nan    
    try:
        train['Владельцы'] = ul.find_all('li', class_='CardInfoRow_ownersCount')[0].find_all('span')[1].text
    except:
        np.nan    
    try:
        train['Владение'] = ul.find_all('li', class_='CardInfoRow_owningTime')[0].find_all('span')[1].text
    except:
        np.nan    
    try:
        train['ПТС'] = ul.find_all('li', class_='CardInfoRow_pts')[0].find_all('span')[1].text
    except:
        np.nan    
    try:
        train['Привод'] = ul.find_all('li', class_='CardInfoRow_drive')[0].find_all('span')[1].text
    except:
        np.nan    
    try:
        train['Руль'] = ul.find_all('li', class_='CardInfoRow_wheel')[0].find_all('span')[1].text
    except:
        np.nan    
    try:
        train['Состояние'] = ul.find_all('li', class_='CardInfoRow_state')[0].find_all('span')[1].text
    except:
        np.nan
    try:
        train['Таможня'] = ul.find_all('li', class_='CardInfoRow_customs')[0].find_all('span')[1].text
    except:
        np.nan
    try:
        train['price'] = script1_json['offers']['price']
    except:
        np.nan 
        
    
    
    df_train = df_train.append([train])

100%|██████████| 17000/17000 [6:10:32<00:00,  1.31s/it]  


In [9]:
df_train.to_csv('df_train1.csv')

In [50]:
df_train = pd.read_csv('df_train1.csv')
df_train2 = pd.read_csv('df_train2.csv')
df_train3 = pd.read_csv('df_train3.csv')
df_train4 = pd.read_csv('df_train4.csv')

In [51]:
df = pd.concat([df_train, df_train2,df_train3, df_train4], ignore_index=True)

In [60]:
# Сохранение финального датафрейма локально

df.to_csv('df_train.csv', index=False)